In [1]:
# GOAL: 
# Given a qasm file content apply the transformation, then run the program 
# on the same platform (QISKIT), which has a perfect reading of QASM files
# and them:
# - run program VS
# - run matamorph(program)

# Metamorphic relationship 
# - 'qubit_order': permute the order of qubits but fix the mapping to HW
# - ''

In [2]:
import pandas as pd
import sys
sys.path.append("../lib")
import seaborn as sns
import matplotlib.pyplot as plt
from utils import iterate_over
from utils import load_config_and_check
from inspector import Inspector
from tqdm import tqdm
import sqlite3 as sl
import os

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from detectors import KS_Detector

In [5]:
#QASM_PATH = "stub_files/example.qasm"
QASM_PATH = "stub_files/easy_example.qasm"
QASM_CONTENT = open(QASM_PATH, 'r').read()

In [6]:
print(QASM_CONTENT)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[0];
cx q[0], q[1];
ry(1.11) q[2];
rz(2.22) q[2];
cx q[2], q[3];
h q[3];
h q[1];
barrier q;
measure q -> c;


In [7]:
import qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile


In [8]:
import re
import uuid
from typing import Dict, Any, List
import numpy as np

def get_n_qubits(qasm_content: str) -> int:
    m = re.search("qreg q\[(\d*)", qasm_content)
    if m:
        return int(m.group(1))
    return -1

def replace_all_reg_occurrences(qasm_content: str, start: str, end: str) -> str:
    qasm_content = qasm_content.replace(f"q[{start}]", f"q[{end}]")
    qasm_content = qasm_content.replace(f"c[{start}]", f"c[{end}]")
    return qasm_content

def scramble_qubits(qasm_content: str, qubits_mapping: Dict[int, int]):
    """Swap the qubits based on order."""
    pairs = [{"start": k, "end": v} for k, v in qubits_mapping.items()]
    triplets = [ {"tmp": uuid.uuid4().hex, **p} for p in pairs]
    print("-" * 80)
    print(qubits_mapping)
    for triplet in triplets:
        qasm_content = replace_all_reg_occurrences(
            qasm_content, start=triplet["start"], end=triplet["tmp"])
    for triplet in triplets:
        qasm_content = replace_all_reg_occurrences(
            qasm_content, start=triplet["tmp"], end=triplet["end"])
    return qasm_content
        
def create_random_mapping(qasm_content: str, seed: int = None) -> Dict[int, int]:
    max_qubits = get_n_qubits(qasm_content)
    if seed: 
        np.random.seed(seed)
    start_qubits = np.arange(max_qubits)
    np.random.shuffle(start_qubits)
    end_qubits = np.arange(max_qubits)
    np.random.shuffle(end_qubits)
    return {f: s for f, s in zip(start_qubits, end_qubits)}
  
def read_str_with_mapping(bitstring: str, direct_mapping: Dict[int, int]):
    """Given a bitstring convert it to the original mapping."""
    n_bits = len(bitstring)
    return "".join([bitstring[direct_mapping[i]] for i in range(n_bits)])
    
def convert_result_to_mapping(result: Dict[str, int], qubits_mapping: Dict[int, int]):
    """Convert the result via the given mapping.
    
    because a qubit maping will make also the result scrambled thus we have
    to reverse the mapping and read the results.
    """
    return {
        read_str_with_mapping(bitstring, qubits_mapping): freq
        for bitstring, freq in result.items()
    }

read_str_with_mapping("011", {0:1, 1:2, 2:0})

'110'

In [9]:
def morph_qubit_order(qasm_path, seed: int = 42):
    """Change the qubit order"""
    content = open(qasm_path, 'r').read()
    print("-" * 80)
    print("BEFORE")
    print(content)
    random_mapping = create_random_mapping(qasm_content=content, seed=seed) 
    new_qasm = scramble_qubits(qasm_content=content, qubits_mapping=random_mapping)
    new_qasm_path = qasm_path + "_morphed"
    with open(new_qasm_path, 'w') as out_file:
        out_file.write(new_qasm)
    print("-" * 80)
    print("AFTER")
    print(new_qasm)
    print(f"Morphed version saved here: {new_qasm_path}")
    new_qc = QuantumCircuit.from_qasm_file(new_qasm_path)
    return new_qc, random_mapping

new_qc, new_mapping = morph_qubit_order(qasm_path=QASM_PATH, seed=14)

--------------------------------------------------------------------------------
BEFORE
OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[0];
cx q[0], q[1];
ry(1.11) q[2];
rz(2.22) q[2];
cx q[2], q[3];
h q[3];
h q[1];
barrier q;
measure q -> c;
--------------------------------------------------------------------------------
{1: 3, 2: 0, 0: 1, 3: 2}
--------------------------------------------------------------------------------
AFTER
OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[1];
cx q[1], q[3];
ry(1.11) q[0];
rz(2.22) q[0];
cx q[0], q[2];
h q[2];
h q[3];
barrier q;
measure q -> c;
Morphed version saved here: stub_files/easy_example.qasm_morphed


In [10]:
new_qc.draw(fold=-1)

┌──────────┐┌──────────┐           ░ ┌─┐         
q_0: ┤ Ry(1.11) ├┤ Rz(2.22) ├──■────────░─┤M├─────────
     └──┬───┬───┘└──────────┘  │        ░ └╥┘┌─┐      
q_1: ───┤ H ├─────────■────────┼────────░──╫─┤M├──────
        └───┘         │      ┌─┴─┐┌───┐ ░  ║ └╥┘┌─┐   
q_2: ─────────────────┼──────┤ X ├┤ H ├─░──╫──╫─┤M├───
                    ┌─┴─┐    ├───┤└───┘ ░  ║  ║ └╥┘┌─┐
q_3: ───────────────┤ X ├────┤ H ├──────░──╫──╫──╫─┤M├
                    └───┘    └───┘      ░  ║  ║  ║ └╥┘
c: 4/══════════════════════════════════════╩══╩══╩══╩═
                                           0  1  2  3

In [11]:
qc = QuantumCircuit.from_qasm_file(QASM_PATH)

In [12]:
qc.draw(fold=-1)

┌───┐                           ░ ┌─┐         
q_0: ───┤ H ├─────────■─────────────────░─┤M├─────────
        └───┘       ┌─┴─┐    ┌───┐      ░ └╥┘┌─┐      
q_1: ───────────────┤ X ├────┤ H ├──────░──╫─┤M├──────
     ┌──────────┐┌──┴───┴───┐└───┘      ░  ║ └╥┘┌─┐   
q_2: ┤ Ry(1.11) ├┤ Rz(2.22) ├──■────────░──╫──╫─┤M├───
     └──────────┘└──────────┘┌─┴─┐┌───┐ ░  ║  ║ └╥┘┌─┐
q_3: ────────────────────────┤ X ├┤ H ├─░──╫──╫──╫─┤M├
                             └───┘└───┘ ░  ║  ║  ║ └╥┘
c: 4/══════════════════════════════════════╩══╩══╩══╩═
                                           0  1  2  3

In [37]:
def execute_circuit(circuit_to_execute: QuantumCircuit, shots: int = 8192, coupling_map=None):
    # Transpile for simulator
    simulator = Aer.get_backend('aer_simulator')
    circ = transpile(circuit_to_execute, simulator, coupling_map=coupling_map)
    print(circ)
    # Run and get counts
    result = simulator.run(circ, shots=shots).result()
    counts = result.get_counts(circ)
    return counts

In [23]:
from qiskit.transpiler import CouplingMap
from itertools import combinations

In [43]:
def generate_random_coupling_map(n_qubits: int = 4, dropout: float = 0.3, seed: int = 42):
    """Create a random qubits connection list for the coupling map."""
    np.random.seed(seed)
    idxs = range(n_qubits)
    all_possible_connections = np.array(list(combinations(idxs, 2)))
    rnd_choice_mask = np.random.choice(
        [True, False], size=len(all_possible_connections), 
        p=[1 - dropout, dropout])
    return list(all_possible_connections[rnd_choice_mask])
    
cm_original = CouplingMap(generate_random_coupling_map(n_qubits=4, seed=42))
cm_new = CouplingMap(generate_random_coupling_map(n_qubits=4, seed=1234))


In [44]:
original_res = execute_circuit(circuit_to_execute=qc, coupling_map=cm_original)
morphed_res = execute_circuit(circuit_to_execute=new_qc, coupling_map=cm_new)
reordered_morphed_res = convert_result_to_mapping(morphed_res, new_mapping)

global phase: 5.1732
                ┌───┐                  ░ ┌─┐         
q_0 -> 0 ───────┤ H ├─────────■────────░─┤M├─────────
                └───┘       ┌─┴─┐┌───┐ ░ └╥┘┌─┐      
q_1 -> 1 ───────────────────┤ X ├┤ H ├─░──╫─┤M├──────
         ┌─────────────────┐└───┘└───┘ ░  ║ └╥┘┌─┐   
q_2 -> 2 ┤ U3(1.11,2.22,0) ├──■────────░──╫──╫─┤M├───
         └─────────────────┘┌─┴─┐┌───┐ ░  ║  ║ └╥┘┌─┐
q_3 -> 3 ───────────────────┤ X ├┤ H ├─░──╫──╫──╫─┤M├
                            └───┘└───┘ ░  ║  ║  ║ └╥┘
    c: 4/═════════════════════════════════╩══╩══╩══╩═
                                          0  1  2  3 
global phase: 5.1732
                     ┌───┐                       ┌───┐   ┌───┐           ░    »
q_2 -> 0 ─────X──────┤ H ├────────────────────■──┤ H ├─X─┤ H ├──■────────░────»
              │      └───┘                    │  └───┘ │ ├───┤  │        ░    »
q_3 -> 1 ─────X───────────────────────────────┼────────X─┤ H ├──┼────────░────»
            ┌───┐         ┌────────────────┐

In [45]:
original_res

{'1110': 271,
 '1100': 274,
 '0100': 293,
 '0001': 731,
 '1111': 256,
 '0000': 746,
 '0010': 782,
 '1011': 774,
 '0011': 703,
 '0111': 268,
 '1000': 740,
 '0110': 293,
 '1101': 295,
 '1010': 720,
 '1001': 738,
 '0101': 308}

In [46]:
morphed_res

{'1100': 752,
 '0010': 708,
 '1011': 289,
 '0111': 303,
 '0011': 285,
 '1110': 728,
 '1001': 290,
 '0101': 283,
 '1111': 259,
 '0001': 285,
 '1010': 734,
 '1101': 283,
 '1000': 675,
 '0110': 808,
 '0000': 727,
 '0100': 783}

In [47]:
reordered_morphed_res

{'1010': 752,
 '0001': 708,
 '0111': 289,
 '1101': 303,
 '0101': 285,
 '1011': 728,
 '0110': 290,
 '1100': 283,
 '1111': 259,
 '0100': 285,
 '0011': 734,
 '1110': 283,
 '0010': 675,
 '1001': 808,
 '0000': 727,
 '1000': 783}

In [52]:
detector = KS_Detector()
stat, p_val_with_scrambled = detector.check(original_res, morphed_res)
print(f"p_val_with_scrambled: {p_val_with_scrambled} [expected: divergence]")
print("Interpretation guide: very low pvalue => divergent programs")

p_val_with_scrambled: 4.36241839563252e-49 [expected: divergence]
Interpretation guide: very low pvalue => divergent programs


In [53]:
detector = KS_Detector()
stat, p_val_with_reordered  = detector.check(original_res, reordered_morphed_res)
print(f"p_val_with_reordered: {p_val_with_reordered} [expected: no divergence]")
print("Interpretation guide: more than 0.05 pvalue => no divergence")

p_val_with_reordered: 0.11915070001671184 [expected: no divergence]
Interpretation guide: more than 0.05 pvalue => no divergence


# Bank of generated programs

In [20]:
PATH_PROGRAMS = "../data/q"

In [21]:
def motamorphic_run():
    pass